<a href="https://colab.research.google.com/github/dfridland/NLP/blob/HW7/NLP_DF_HW7_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gensim
import pandas as pd
from sklearn.model_selection import train_test_split

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from nltk.probability import FreqDist

import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.callbacks import TensorBoard 
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import EarlyStopping  

from sklearn.preprocessing import LabelEncoder

from pathlib import Path

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/evamelissatasdemir/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#f-score
import keras.backend as K

def get_f1(y_true, y_pred): #taken from old keras source code
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    
    return f1_val

In [ ]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Загружаем данные
df = pd.read_excel('./lection7/отзывы за лето.xls')
df.head(10)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
5,5,Всё удобно норм 👍👍👍,2017-08-14
6,5,Очень удобное приложение.,2017-08-14
7,5,Все устраивает,2017-08-14
8,5,У меня работает все четко. В отличии от банком...,2017-08-14
9,5,Очень все хорошо👍,2017-08-14


In [ ]:
df['Rating'].value_counts()

Rating
5    14586
1     2276
4     2138
3      911
2      748
Name: count, dtype: int64

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

df_train.shape, df_test.shape

((13841, 3), (6818, 3))

In [ ]:
df_train

,Rating,Content,Date
0,5,Наконец-то исправили эту чушь с неоргинальной ...,2017-08-09
1,5,Удобно в использовании,2017-07-27
2,5,Отлично,2017-08-08
3,5,Класс,2017-07-25
4,5,Удобно,2017-07-08
...,...,...,...
13836,4,Все нравится,2017-07-29
13837,5,Очень смешно программа пугается рута :),2017-07-28
13838,1,Не могу скачать ошибка номер 24,2017-08-06
13839,5,Сбербанк всегда рядом,2017-08-12


In [ ]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [ ]:
# Creating dictionary
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()
tokens = word_tokenize(train_corpus)

In [ ]:
# filter and take only top N токенов
tokens_filtered = [word for word in tokens if word.isalnum()] #filter to keep only letters and numbers

# create token dictionary: number in corpus
dist = FreqDist(tokens_filtered)
dist

FreqDist({'приложение': 4123, 'удобно': 2201, 'работать': 1288, 'удобный': 1182, 'отлично': 860, 'нравиться': 763, 'хороший': 681, 'отличный': 677, 'телефон': 627, 'супер': 540, ...})

In [ ]:
max_words = 200


tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [ ]:
# create a vocabulary from top-200 tokens 

vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}


In [ ]:
max_len = 40

# preprocess train and test datasets 
# to lower register, tokenize, remove non number and non letter symbols
# Check if token is in dictionary - add token's number to the result
#otherwise - miss it and add zeros till the max_len

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [ ]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)
x_train

array([[  0,   0,   0, ...,   1,   2,  15],
       [  0,   0,   0, ...,   0,   2, 181],
       [  0,   0,   0, ...,   0,   0,   5],
       ...,
       [  0,   0,   0, ..., 164,  27,  84],
       [  0,   0,   0, ...,   0,   0,  20],
       [  0,   0,   0, ...,   0, 113,   5]], dtype=int32)

In [ ]:
x_train.shape

(13841, 40)

In [ ]:
#one-hot encoding
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating']) 
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5])

In [ ]:
train_enc_labels

array([4, 4, 4, ..., 0, 4, 4])

In [ ]:
num_classes = 5
y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)
y_train

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [ ]:
## Keras  CONV model with preset Embedding

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=300, input_length=max_len))
model.add(Conv1D(72, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D()) #choose the right pooling or you can use dense layer or rnn instead if accuracy is low
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[get_f1])

In [ ]:
epochs = 20
batch_size = 512

In [ ]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss') 

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 1s 21ms/step - loss: 1.5222 - get_f1: 0.0000e+00 - val_loss: 1.5020 - val_get_f1: 0.0000e+00
Epoch 2/20
25/25 [==============================] - 0s 20ms/step - loss: 1.4836 - get_f1: 0.0000e+00 - val_loss: 1.4637 - val_get_f1: 0.0000e+00
Epoch 3/20
25/25 [==============================] - 1s 20ms/step - loss: 1.4450 - get_f1: 0.0000e+00 - val_loss: 1.4248 - val_get_f1: 0.0000e+00
Epoch 4/20
25/25 [==============================] - 0s 20ms/step - loss: 1.4054 - get_f1: 0.0000e+00 - val_loss: 1.3846 - val_get_f1: 0.0000e+00
Epoch 5/20
25/25 [==============================] - 0s 20ms/step - loss: 1.3644 - get_f1: 0.0000e+00 - val_loss: 1.3429 - val_get_f1: 0.0000e+00
Epoch 6/20
25/25 [==============================] - 0s 19ms/step - loss: 1.3217 - get_f1: 0.0000e+00 - val_loss: 1.2997 - val_get_f1: 0.0000e+00
Epoch 7/20
25/25 [==============================] - 0s 20ms/step - loss: 1.2782 - get_f1: 0.0000e+00 - val_loss: 1.2565 - val_get_

In [ ]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test f1_score:', score[1])

14/14 [==============================] - 0s 13ms/step - loss: 0.9826 - get_f1: 0.7151


Test loss: 0.9826066493988037
Test f1_score: 0.7151089310646057


In [ ]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
#!unzip glove*.zip


In [ ]:
!ls
!pwd

180.zip
180.zip.1
NLP_DF_HW7.ipynb
NLP_DF_HW7_1.ipynb
README
glove.6B.100d.txt
glove.6B.200d.txt
glove.6B.300d.txt
glove.6B.50d.txt
glove.6B.zip
glove.6B.zip.1
hw7.txt
lection7
lection7.zip
logs
meta.json
model.bin
model.txt
Пример_разбора_ДЗ_NLP_07_1.ipynb
/Users/evamelissatasdemir/NLP/Lesson7


In [ ]:
word_index = keras.datasets.imdb.get_word_index()

In [ ]:

# embeddings_index = {}
# f = open('glove.6B.100d.txt')
# for line in f:
#     values = line.split() # seperates each line by a white space
#     word = values[0]  # the first element is the word
#     coefs = np.asarray(values[1:], dtype='float32')  # the rest are the weights
#     # storing in dictionary
#     embeddings_index[word] = coefs
# f.close()

# print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
embeddings_dictionary = dict()

In [ ]:
glove_file = open('./glove.6B.300d.txt', 'rb')
for line in glove_file:
    records = line.split()  # seperates each line by a white space
    word = records[0]  # the first element is the word
    vector_dimensions = np.asarray(
        records[1:], dtype='float32')  # the rest are the weights
    # storing in dictionary
    embeddings_dictionary[word] = vector_dimensions
    
glove_file.close()
print('Found %s word vectors.' % len(embeddings_dictionary))

Found 400000 word vectors.


In [ ]:
# len_of_vocab = len(word_index)
embeddings_matrix = np.zeros((max_words , 300))
# mapping to a new matrix, using only the words in your tokenizer's vocabulary
for word, index in word_index.items():
    if index>=max_words:
        continue
    # the weights of the individual words in your vocabulary
    embedding_vector = embeddings_dictionary.get(bytes(word, 'utf-8'))
 
    if embedding_vector is not None:
        embeddings_matrix[index] = embedding_vector
        #embeddings_matrix[index] = embedding_vector
        



In [ ]:
initializer = tf.keras.initializers.Constant(embeddings_matrix[:200])

model1 = Sequential()
model1.add(Embedding(input_dim=max_words, output_dim=300,
          input_length=max_len, weights=[embeddings_matrix], embeddings_initializer =initializer, trainable=True))
#model.add(Embedding(input_dim=max_words, output_dim=128, embeddings_initializer =initializer, input_length=max_len, trainable=False))
model1.add(Conv1D(128, 3))
model1.add(Activation("relu"))
model1.add(GlobalMaxPool1D())
model1.add(Dense(10))
model1.add(Activation("relu"))
model1.add(Dense(num_classes))
model1.add(Activation('softmax'))

In [ ]:
# Log to tensorboard
tensorBoardCallback = TensorBoard(log_dir='./logs', write_graph=True)
model1.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=[get_f1])


model1.fit(x_train, y_train, epochs=3, callbacks=[
          tensorBoardCallback], batch_size=64)


# Evaluation on the test set
scores = model1.evaluate(x_test, y_test, verbose=0)
print("f1 score: %.2f%%" % (scores[1]*100))

Epoch 1/3
217/217 [==============================] - 4s 19ms/step - loss: 0.7679 - get_f1: 0.7211
Epoch 2/3
217/217 [==============================] - 3s 15ms/step - loss: 0.6666 - get_f1: 0.7713
Epoch 3/3
217/217 [==============================] - 3s 16ms/step - loss: 0.6351 - get_f1: 0.7838
f1 score: 76.59%


In [ ]:
score = model1.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', scores[0])
print('Test f1:', scores[1])

14/14 [==============================] - 0s 14ms/step - loss: 0.6733 - get_f1: 0.7738


Test loss: 0.6732770204544067
Test f1: 0.7706745266914368


In [ ]:
# Conclusion 

# I achieved better metrics using pre-trained word embeddings in a Keras model (f1_score = 0.77067)
# 3) than Keras embadding (f1_score: 0.7151). Loss is less using glove pre-trained word embeddings(Test loss: 0.67327)
# than Keras embadding(Test loss:0.9826)


In [ ]:
#import gensim.downloader as api

In [ ]:
#from gensim.models import Word2Vec

In [ ]:
#model=gensim.models.Word2Vec(sentences) with model=Word2Vec(sentences)